# WAMEX reports: pre-process data and train the embedding with FastText

In [1]:
from gensim.models import FastText

import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

from stop_words import get_stop_words
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

stop_words = list(get_stop_words('en'))         # 174 stopwords
nltk_words = list(stopwords.words('english'))   # 153 stopwords
stop_words.extend(nltk_words)                   # 353 in total

import re
import os, glob, codecs
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
logging.info('START - FastText Embeddings for all wamex reports')

WAMEX_DATA_FOLDER = r"/Users/majiga/Documents/wamex/data/wamex_xml"

MODEL_FILE = r"Vectors/fastText_wamex_raw.model" # bin file

2018-11-09 13:17:16,378 : INFO : START - FastText Embeddings for all wamex reports


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/majiga/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/majiga/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/majiga/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## Load a trained model

In [3]:
# load model
model_fasttext = FastText.load(MODEL_FILE)
print(new_model)

model_fasttext.wv.most_similar('gold')

2018-11-09 13:23:13,426 : INFO : loading FastText object from Vectors/fastText_wamex_raw.model
2018-11-09 13:23:13,755 : INFO : loading wv recursively from Vectors/fastText_wamex_raw.model.wv.* with mmap=None
2018-11-09 13:23:13,756 : INFO : setting ignored attribute vectors_norm to None
2018-11-09 13:23:13,757 : INFO : setting ignored attribute vectors_vocab_norm to None
2018-11-09 13:23:13,757 : INFO : setting ignored attribute vectors_ngrams_norm to None
2018-11-09 13:23:13,757 : INFO : setting ignored attribute buckets_word to None
2018-11-09 13:23:13,758 : INFO : loading vocabulary recursively from Vectors/fastText_wamex_raw.model.vocabulary.* with mmap=None
2018-11-09 13:23:13,758 : INFO : loading trainables recursively from Vectors/fastText_wamex_raw.model.trainables.* with mmap=None
2018-11-09 13:23:13,758 : INFO : loaded Vectors/fastText_wamex_raw.model
2018-11-09 13:23:13,774 : INFO : precomputing L2-norms of word weight vectors
2018-11-09 13:23:13,778 : INFO : precomputing L

FastText(vocab=8562, size=100, alpha=0.025)


[('au', 0.7181254625320435),
 ('copper', 0.6624714732170105),
 ('nickel', 0.6557887196540833),
 ('metal', 0.6335586905479431),
 ('precious', 0.6275296807289124),
 ('arsenic', 0.6186712384223938),
 ('antimony', 0.6167358756065369),
 ('tungsten', 0.6000398993492126),
 ('historically', 0.5887951850891113),
 ('resolute', 0.5784502029418945)]

In [4]:
# model_fasttext.wv.most_similar('kalgoorlie')

print(model_fasttext.wv.most_similar('iron-ore'))

print(model_fasttext.wv.most_similar('iron', topn=10))


print(model_fasttext.wv.most_similar(positive=['kalgoorlie','iron-ore'],negative=['gold']))

print(new_model.wv['gold'])  # numpy vector of a word

[('iron', 0.7587395310401917), ('ironcap', 0.6327366828918457), ('marra', 0.5913161039352417), ('brockman', 0.5766406655311584), ('bif', 0.5754503607749939), ('sinosteel', 0.5720453262329102), ('nammuldi', 0.5704472661018372), ('manganese', 0.5699800252914429), ('banded', 0.5622855424880981), ('finnerty', 0.5618066787719727)]
[('marra', 0.6866266131401062), ('brockman', 0.6819794774055481), ('hematite', 0.6760455369949341), ('banded', 0.6720453500747681), ('detritals', 0.6715987920761108), ('bid', 0.6652620434761047), ('bif', 0.6647725701332092), ('ore', 0.6554528474807739), ('manganese', 0.6512892842292786), ('dso', 0.6481667160987854)]
[('esperance', 0.47153985500335693), ('bunbury', 0.45931476354599), ('pannawonica', 0.45913761854171753), ('karratha', 0.44624924659729004), ('geraldton', 0.4430530369281769), ('coolgardie', 0.42476966977119446), ('balladonia', 0.42342591285705566), ('vetters', 0.41675683856010437), ('hyden', 0.4077901244163513), ('bulong', 0.404255747795105)]
[ 0.0562

In [5]:
words = list(model_fasttext.wv.vocab)
print(len(words))

8562


## Create embeddings
Note: Or load already trained model